In [1]:
# Perturbations traffic
# Ascenseurs

In [2]:
# Installation des dépendances :
!pip install s3fs

# import des dépendances
import os
import s3fs

# Creation d'un objet filesystem, les variables d'environnement on été automatiquement traitées par Onyxia
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

# Liste des documents du répertoire partagé dédié au hackathon :
fs.ls("dlb-hackathon/datasets-diffusion", refresh=True)

PermissionError: The Access Key Id you provided does not exist in our records.

In [3]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [4]:
# Lecture d'un fichier csv et chargement en DataFrame Panda : 
import pandas as pd

BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "equipe-2/etat-des-ascenseurs.parquet"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    etat_ascenseurs = pd.read_parquet(file_in)
etat_ascenseurs

,zdcid,zdcxepsg2154,zdcyepsg2154,liftid,liftreason,liftstatus,privateelevatorid,liftsituation,liftdirection,liftstateupdate,liftstate,zdcname,severity,liftmode,centroidzdc
0,71485,644961,6865544,318,,available,PMR 1 AQ-DEFESP1,Surface - Salle d échange - Quai,La Défense - Grande Arche,2024-11-21T14:11,1,Esplanade de la Défense,1.0,Metro,b'\x01\x01\x00\x00\x00\xf7\x97\x86+\xd2\xfe\x0...
1,70441,649888,6856847,1038,,available,ASC 4 - EQ11-A005-000000301,SA200,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
2,70441,649888,6856847,1039,,available,ASC 5 - EQ11-A005-000000302,SA200,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
3,70441,649888,6856847,1040,,available,ASC 1 - EQ11-A005-000000305,CR212,Bagneux – Lucie Aubrac,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
4,70441,649888,6856847,1042,,available,ASC 7 - EQ11-A005-000000304,SA217,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,62667,649706,6843023,1156,,available,393645 - ASC.PMR 4577,Quai 2,None,2024-11-21T14:20,1,Gravigny Balizy,1.0,Tramway,b'\x01\x01\x00\x00\x00-Z\t^\xc7\x88\x02@\xde\x...
935,62977,648140,6844987,1152,,available,393611 - ASC.PMR 4566,Quai 2,None,2024-11-21T14:20,1,Longjumeau,1.0,Tramway,b'\x01\x01\x00\x00\x00\xa2\xbd-\xda\xb9\\\x02@...
936,63244,645528,6847403,479,,available,4211,Sortie MASSY VILMORIN,None,2024-11-21T14:22,1,Massy - Palaiseau,1.0,LocalTrain,b'\x01\x01\x00\x00\x00\xbd\xdb\x069h\x13\x02@S...
937,63244,645528,6847403,488,/,unkown,MAPA PMR7-CR237,Voirie (rue Carnot),None,2024-11-18T15:30,0,Massy - Palaiseau,1.0,LocalTrain,b'\x01\x01\x00\x00\x00\xbd\xdb\x069h\x13\x02@S...


In [ ]:
import duckdb as ddb
ddb.execute("SET s3_region='fr-central';")
ddb.execute("SET s3_url_style='path';")
ddb.execute("SET s3_endpoint='minio.data-platform-self-service.net';")
ddb.execute(f"SET s3_access_key_id='{os.environ["AWS_ACCESS_KEY_ID"]}' ;") # Aussi récupérable dans les paramètres "Valeurs de Helm" du service
ddb.execute(f"SET s3_secret_access_key='{os.environ["AWS_SECRET_ACCESS_KEY"]}';") # Aussi récupérable dans les paramètres "Valeurs de Helm" 

In [6]:
equipement_id = 1004
limit_date = (datetime.now() - relativedelta(month=6)).strftime("%Y-%m-%d %H:%M:%S")

history_ascenseur = ddb.sql(f""" 
        select * from 's3://dlb-hackathon/datasets-diffusion/ascenseurs_historique_etat/RELEVES_ETATS_ASCENSEURS_SNCF_RATP_2021-2024.parquet'
        where code_appareil = {equipement_id}
        and date_releve >= '{limit_date}'
        order by date_releve DESC LIMIT 1
        """).to_df()
history_ascenseur # 
        # and date_releve >= '{limit_date}'

,operateur,date_releve,code_appareil,code_fonctionnel,etat,date_etat_debut,date_etat_fin,emplacement_gare,emplacement_situation,emplacement_direction
0,RATP,2024-10-28 19:08:38,1004,ASC01-EQ11-A005-000000269,1,28/10/2024,None,SAINT OUEN,SA200,None


In [7]:
BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "equipe-2/grievances.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="r") as file_in:
    grievances = pd.read_csv(file_in, index_col=0)

In [8]:
from add_state_to_equipements import get_state_from_grievances

In [9]:
etat_ascenseurs

,zdcid,zdcxepsg2154,zdcyepsg2154,liftid,liftreason,liftstatus,privateelevatorid,liftsituation,liftdirection,liftstateupdate,liftstate,zdcname,severity,liftmode,centroidzdc
0,71485,644961,6865544,318,,available,PMR 1 AQ-DEFESP1,Surface - Salle d échange - Quai,La Défense - Grande Arche,2024-11-21T14:11,1,Esplanade de la Défense,1.0,Metro,b'\x01\x01\x00\x00\x00\xf7\x97\x86+\xd2\xfe\x0...
1,70441,649888,6856847,1038,,available,ASC 4 - EQ11-A005-000000301,SA200,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
2,70441,649888,6856847,1039,,available,ASC 5 - EQ11-A005-000000302,SA200,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
3,70441,649888,6856847,1040,,available,ASC 1 - EQ11-A005-000000305,CR212,Bagneux – Lucie Aubrac,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
4,70441,649888,6856847,1042,,available,ASC 7 - EQ11-A005-000000304,SA217,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,62667,649706,6843023,1156,,available,393645 - ASC.PMR 4577,Quai 2,None,2024-11-21T14:20,1,Gravigny Balizy,1.0,Tramway,b'\x01\x01\x00\x00\x00-Z\t^\xc7\x88\x02@\xde\x...
935,62977,648140,6844987,1152,,available,393611 - ASC.PMR 4566,Quai 2,None,2024-11-21T14:20,1,Longjumeau,1.0,Tramway,b'\x01\x01\x00\x00\x00\xa2\xbd-\xda\xb9\\\x02@...
936,63244,645528,6847403,479,,available,4211,Sortie MASSY VILMORIN,None,2024-11-21T14:22,1,Massy - Palaiseau,1.0,LocalTrain,b'\x01\x01\x00\x00\x00\xbd\xdb\x069h\x13\x02@S...
937,63244,645528,6847403,488,/,unkown,MAPA PMR7-CR237,Voirie (rue Carnot),None,2024-11-18T15:30,0,Massy - Palaiseau,1.0,LocalTrain,b'\x01\x01\x00\x00\x00\xbd\xdb\x069h\x13\x02@S...


In [10]:
from add_state_to_equipements import compute_state_for_elevators

In [11]:
etat_ascenseurs

,zdcid,zdcxepsg2154,zdcyepsg2154,liftid,liftreason,liftstatus,privateelevatorid,liftsituation,liftdirection,liftstateupdate,liftstate,zdcname,severity,liftmode,centroidzdc
0,71485,644961,6865544,318,,available,PMR 1 AQ-DEFESP1,Surface - Salle d échange - Quai,La Défense - Grande Arche,2024-11-21T14:11,1,Esplanade de la Défense,1.0,Metro,b'\x01\x01\x00\x00\x00\xf7\x97\x86+\xd2\xfe\x0...
1,70441,649888,6856847,1038,,available,ASC 4 - EQ11-A005-000000301,SA200,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
2,70441,649888,6856847,1039,,available,ASC 5 - EQ11-A005-000000302,SA200,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
3,70441,649888,6856847,1040,,available,ASC 1 - EQ11-A005-000000305,CR212,Bagneux – Lucie Aubrac,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
4,70441,649888,6856847,1042,,available,ASC 7 - EQ11-A005-000000304,SA217,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,62667,649706,6843023,1156,,available,393645 - ASC.PMR 4577,Quai 2,None,2024-11-21T14:20,1,Gravigny Balizy,1.0,Tramway,b'\x01\x01\x00\x00\x00-Z\t^\xc7\x88\x02@\xde\x...
935,62977,648140,6844987,1152,,available,393611 - ASC.PMR 4566,Quai 2,None,2024-11-21T14:20,1,Longjumeau,1.0,Tramway,b'\x01\x01\x00\x00\x00\xa2\xbd-\xda\xb9\\\x02@...
936,63244,645528,6847403,479,,available,4211,Sortie MASSY VILMORIN,None,2024-11-21T14:22,1,Massy - Palaiseau,1.0,LocalTrain,b'\x01\x01\x00\x00\x00\xbd\xdb\x069h\x13\x02@S...
937,63244,645528,6847403,488,/,unkown,MAPA PMR7-CR237,Voirie (rue Carnot),None,2024-11-18T15:30,0,Massy - Palaiseau,1.0,LocalTrain,b'\x01\x01\x00\x00\x00\xbd\xdb\x069h\x13\x02@S...


In [12]:
compute_state_for_elevators(etat_ascenseurs)

,zdcid,zdcxepsg2154,zdcyepsg2154,liftid,liftreason,liftstatus,privateelevatorid,liftsituation,liftdirection,liftstateupdate,liftstate,zdcname,severity,liftmode,centroidzdc,state,state_grievances,last_datetime_grievances
0,71485,644961,6865544,318,,available,PMR 1 AQ-DEFESP1,Surface - Salle d échange - Quai,La Défense - Grande Arche,2024-11-21T14:11,1,Esplanade de la Défense,1.0,Metro,b'\x01\x01\x00\x00\x00\xf7\x97\x86+\xd2\xfe\x0...,3.0,3.0,2024-11-22 09:11:31.888528
1,70441,649888,6856847,1038,,available,ASC 4 - EQ11-A005-000000301,SA200,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...,3.0,3.0,2024-11-22 09:20:51.194811
2,70441,649888,6856847,1039,,available,ASC 5 - EQ11-A005-000000302,SA200,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...,0.0,0.0,NaN
3,70441,649888,6856847,1040,,available,ASC 1 - EQ11-A005-000000305,CR212,Bagneux – Lucie Aubrac,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...,0.0,0.0,NaN
4,70441,649888,6856847,1042,,available,ASC 7 - EQ11-A005-000000304,SA217,Voirie – Mezzanine > toutes directions,2024-11-21T14:11,1,Barbara,1.0,Metro,b'\x01\x01\x00\x00\x00F\x92\xb1\xba\x84\x8a\x0...,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,62667,649706,6843023,1156,,available,393645 - ASC.PMR 4577,Quai 2,None,2024-11-21T14:20,1,Gravigny Balizy,1.0,Tramway,b'\x01\x01\x00\x00\x00-Z\t^\xc7\x88\x02@\xde\x...,3.0,3.0,2024-11-22 09:11:31.904804
935,62977,648140,6844987,1152,,available,393611 - ASC.PMR 4566,Quai 2,None,2024-11-21T14:20,1,Longjumeau,1.0,Tramway,b'\x01\x01\x00\x00\x00\xa2\xbd-\xda\xb9\\\x02@...,0.0,0.0,NaN
936,63244,645528,6847403,479,,available,4211,Sortie MASSY VILMORIN,None,2024-11-21T14:22,1,Massy - Palaiseau,1.0,LocalTrain,b'\x01\x01\x00\x00\x00\xbd\xdb\x069h\x13\x02@S...,0.0,0.0,NaN
937,63244,645528,6847403,488,/,unkown,MAPA PMR7-CR237,Voirie (rue Carnot),None,2024-11-18T15:30,0,Massy - Palaiseau,1.0,LocalTrain,b'\x01\x01\x00\x00\x00\xbd\xdb\x069h\x13\x02@S...,NaN,NaN,NaN


In [13]:
BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "equipe-2/ratp_localisation_escaliers.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3
with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    escaliers = pd.read_csv(file_in)


In [14]:
escaliers

,nom,id_escalier,type_esc,nb_marche,rampe,etage,etage.1,transport,compatibilite_pmr,localisation
0,Neuilly-Plaisance,1681522,EM,0.0,OUI,2,2.0,R,Non,POINT (1664329.3825000064 8183834.528700936)
1,Neuilly-Plaisance,1681521,EF,0.0,NON,2,NaN,NaN,NaN,POINT (1664335.1291831804 8183820.698352249)
2,Neuilly-Plaisance,1681517,EF,0.0,OUI,2,NaN,NaN,NaN,POINT (1664329.7091644898 8183830.236635406)
3,Neuilly-Plaisance,1681503,EF,0.0,OUI,2,NaN,NaN,NaN,POINT (1664332.388744174 8183825.695715474)
4,Neuilly-Plaisance,1681523,EF,0.0,OUI,2,NaN,NaN,NaN,POINT (1664342.460116976 8183809.116096983)
...,...,...,...,...,...,...,...,...,...,...
7930,République,100179257,EF,18.0,NON,-1,NaN,NaN,NaN,POINT (1653305.279449177 8185413.0013774475)
7931,République,100179184,EF,20.0,NON,-1,NaN,NaN,NaN,POINT (1653387.3428411044 8185506.632547693)
7932,République,100179253,EF,20.0,OUI,-1,NaN,NaN,NaN,POINT (1653378.98230757 8185502.1184095945)
7933,République,100179179,EF,17.0,NON,-4,NaN,NaN,NaN,POINT (1653266.7856316722 8185482.763964624)


In [16]:
from add_state_to_equipements import compute_state_for_escaliers

ImportError: cannot import name 'compute_state_for_escaliers' from 'add_state_to_equipements' (/home/onyxia/work/accessibility-waze/add_state_to_equipements.py)

In [ ]:
compute_state_for_escaliers()